<a href="https://colab.research.google.com/github/guillem-ms/IA-RM/blob/main/Detectron2vsYOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detectron2 Vs YOLOv5

After training both models, we test and compare them.

<a target="_blank"  href="https://colab.research.google.com/drive/15xa6kVtLJNeWKDKmEN0pF5YLm6h8yJ9C#scrollTo=wPvaEIlxq_Bd">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>

In [1]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 28.7MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=4f12f4f0a66ca7a33f6e7d92071bf808344f1ade436d8eb37792944831d00247
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.8.1+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.2MB 609kB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-cp37-none-any.whl size=58543 sha256=558f15540d16f87bc4aee7159327cdd977bacb68db55368ad547835fae45570b
  Stored in directory: /root/.cache/pip/wheels/d2/ee/3a/5c531df777c03d8c67f22c65f97d6f75321087482d05a9b218
Successfully built fvcore


In [3]:
#Mount drive so we can upload final_models

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%capture
!unzip /content/drive/MyDrive/Models/model_final.zip


In [5]:
%%capture
!curl -L "https://app.roboflow.com/ds/vLQMC20FNt?key=6DUxyj946U" > roboflow.zip; unzip -o roboflow.zip; rm roboflow.zip

In [6]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [7]:
from detectron2.data.datasets import register_coco_instances

for d in ["test"]:
    register_coco_instances(f"simObjects_{d}", {}, f"/content/{d}/_annotations.coco.json", f"/content/{d}")

In [8]:
import random
from detectron2.data import DatasetCatalog, MetadataCatalog
from google.colab.patches import cv2_imshow

dataset_dicts = DatasetCatalog.get("simObjects_test")
simObjects_train_metadata = MetadataCatalog.get("simObjects_test")

WARNING [05/30 17:21:23 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/30 17:21:23 d2.data.datasets.coco]: Loaded 18 images in COCO format from /content/test/_annotations.coco.json


In [9]:
from detectron2.config import get_cfg
import os

cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = "model_final.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
predictor = DefaultPredictor(cfg)

In [10]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS =  "model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
evaluator = COCOEvaluator("simObjects_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "simObjects_test")
inference_on_dataset(predictor.model, val_loader, evaluator)

WARNING [05/30 17:21:38 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [05/30 17:21:38 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/30 17:21:38 d2.data.datasets.coco]: Loaded 18 images in COCO format from /content/test/_annotations.coco.json
[05/30 17:21:38 d2.data.build]: Distribution of instances among all 10 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
| Coppelia-Si.. | 0            |     Apple     | 5            |    Banana     | 3            |
| BluePillsGl.. | 8            | BluePurpleP.. | 5            |     Glass     | 3            |
| GreenBlackP.. | 3            |    Orange     | 6            | RedPillsGlass | 6            |
|  TV Command   | 3 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[05/30 17:21:39 d2.evaluation.evaluator]: Inference done 11/18. 0.1051 s / img. ETA=0:00:00
[05/30 17:21:40 d2.evaluation.evaluator]: Total inference time: 0:00:01.458028 (0.112156 s / img per device, on 1 devices)
[05/30 17:21:40 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:01 (0.104918 s / img per device, on 1 devices)
[05/30 17:21:40 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/30 17:21:40 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[05/30 17:21:40 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[05/30 17:21:40 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[05/30 17:21:40 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.01 seconds.
[05/30 17:21:40 d2.evaluation.fast_eval_api]: Accumulating evaluation results...
[05/30 17:21:40 d2.evaluation.fa

OrderedDict([('bbox',
              {'AP': 66.41940384514642,
               'AP-Apple': 87.02970297029702,
               'AP-Banana': 83.36633663366337,
               'AP-BluePillsGlass': 45.79207920792079,
               'AP-BluePurplePillGlass': 61.18811881188119,
               'AP-Coppelia-Simulation-Objects': nan,
               'AP-Glass': 59.7029702970297,
               'AP-GreenBlackPillsGlass': 55.816831683168324,
               'AP-Orange': 80.48090523338048,
               'AP-RedPillsGlass': 64.42244224422443,
               'AP-TV Command': 59.975247524752476,
               'AP50': 80.55424590078056,
               'AP75': 80.55424590078056,
               'APl': 49.991749174917494,
               'APm': 70.60275223950966,
               'APs': nan})])

# YOLOv5

In [11]:
%%capture
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

In [12]:
%%capture
!curl -L "https://app.roboflow.com/ds/o032xnyOfQ?key=F1k7Kq0q2j" > roboflow.zip; unzip -o roboflow.zip; rm roboflow.zip

In [13]:
!rm data.yaml

In [14]:
!unzip /content/drive/MyDrive/Models/YOLOv5.zip

Archive:  /content/drive/MyDrive/Models/YOLOv5.zip
  inflating: best.pt                 
  inflating: data.yaml               


In [15]:
# Run YOLOv5x on
!python test.py --weights best.pt --data data.yaml --img 512 --iou 0.65 --task test

Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data.yaml', device='', exist_ok=False, img_size=512, iou_thres=0.65, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='test', verbose=False, weights=['best.pt'])
YOLOv5 🚀 v5.0-128-g4b52e19 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 224 layers, 7075486 parameters, 0 gradients, 16.4 GFLOPS
test: Scanning 'test/labels' images and labels... 18 found, 0 missing, 0 empty, 0 corrupted: 100% 18/18 [00:00<00:00, 1637.51it/s]
test: New cache created: test/labels.cache
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100% 1/1 [00:00<00:00,  1.78it/s]
                 all          18          42       0.831       0.945       0.876       0.762
               Apple          18           5       0.811           1       0.929       0.819
              Banana          18           3